SCRIPT TO GENERATE FERTILITY TRAITS
- NINS: Number of inseminations
- CFI: Interval from calving to first service
- CLI: Interval from calving to last service
- FLI: Interval from first to last service
- CI: Calving interval
- GL: Gestation length
- CR: Conception rate

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#
#
# FERTILITY TRAITS PREPARATIONS
df_ins = pd.read_csv("updateDF.csv", low_memory=False)
col_keep = ["SE_Number", "LactationNumber", "CalvingDate"]
df_ins2a = df_ins[col_keep]
df_ins2a = df_ins2a.drop_duplicates(subset=["SE_Number", "LactationNumber", "CalvingDate"])

# Make next_calving
df_ins2b = df_ins2a.drop_duplicates(subset=["SE_Number", "LactationNumber", "CalvingDate"])
df_ins2b = df_ins2b.copy()
df_ins2b["next_calving"] = df_ins2b.groupby(["SE_Number"])["CalvingDate"].shift(-1)
df_ins2c = df_ins2a.merge(df_ins2b, on=["SE_Number", "LactationNumber", "CalvingDate"],
                          how="left")
"""SE-169e580a-2831
SE_Number = ["SE-064c0cec-1189"]
df_ins2c = df_ins2c[df_ins2c["SE_Number"].isin(SE_Number)]
"""
df_ins2c.to_csv("fertility1.csv", index=False)

# Add to original dataframe
df_ins2d = df_ins.merge(df_ins2c, on=["SE_Number", "LactationNumber", "CalvingDate"], how="left")
"""
SE_Number = ["SE-064c0cec-1189"]
df_ins2d = df_ins2d[df_ins2d["SE_Number"].isin(SE_Number)]
"""
df_ins2d.to_csv("fertility2.csv", index=False)

In [ ]:
# NINS - NUMBER OF INSEMINATIONS
ins_count = df_ins2d.drop_duplicates(subset=["SE_Number", "LactationNumber", "InseminationDate"])
ins_count = (ins_count.groupby(["SE_Number", 'LactationNumber']).size().reset_index(name='NINS'))
df_ins2d = df_ins2d.merge(ins_count, on=["SE_Number", "LactationNumber"], how="left")
df_ins2d.to_csv("fertility2.csv", index=False)

In [ ]:
# CFI - INTERVAL FROM CALVING TO FIRST SERVICE
# CLI - INTERVAL FROM CALVING TO LAST SERVICE
# FLI - INTERVAL FROM FIRST TO LAST SERVICE
# Group by cow and lactation, and get the first and last inseminations
first_observations = (df_ins2d.groupby(["SE_Number", "LactationNumber"]).first().reset_index())
last_observations = (df_ins2d.groupby(["SE_Number", "LactationNumber"]).last().reset_index())

print(last_observations.columns)
print(len(last_observations.columns))

col_keep = ["SE_Number", "LactationNumber", "CalvingDate", "InseminationDate"]
first_observations = first_observations[col_keep]

col_keep = ["SE_Number", "LactationNumber", "CalvingDate", "InseminationDate"]
last_observations = last_observations[col_keep]

# Rename the columns of the last observations df to distinguish them
last_observations.rename(columns={
    "CalvingDate": "CalvingDate_last",
    "InseminationDate": "InseminationDate_last",
}, inplace=True)


# Concatenate first and last observations df side by side
df_ins2e = pd.concat([first_observations, last_observations.iloc[:, 2:]], axis=1)

# Convert columns to datetime objects
df_ins2e["InseminationDate"] = pd.to_datetime(df_ins2e["InseminationDate"])
df_ins2e["InseminationDate_last"] = pd.to_datetime(df_ins2e["InseminationDate_last"])
df_ins2e["CalvingDate"] = pd.to_datetime(df_ins2e["CalvingDate"])
df_ins2e["CalvingDate_last"] = pd.to_datetime(df_ins2e["CalvingDate_last"])

# Calculate fertility traits
df_ins2e["CFI"] = (df_ins2e["InseminationDate"] - df_ins2e["CalvingDate"]).dt.days
df_ins2e["CLI"] = (df_ins2e["InseminationDate_last"] - df_ins2e["CalvingDate_last"]).dt.days
df_ins2e["FLI"] = (df_ins2e["InseminationDate_last"] - df_ins2e["InseminationDate"]).dt.days
col_keep = ["SE_Number", "LactationNumber", "CFI", "CLI", "FLI", "InseminationDate_last"]
df_ins2e = df_ins2e[col_keep]

df_ins2f = df_ins2d.merge(df_ins2e, on=["SE_Number", "LactationNumber"], how="left")
df_ins2f.to_csv("fertility3.csv", index=False)

In [ ]:
# CI - CALVING INTERVAL
df_ins2g = df_ins2f.drop_duplicates(subset=["SE_Number", "LactationNumber", "CalvingDate", "next_calving"])
col_keep = ["SE_Number", "LactationNumber", "CalvingDate", "next_calving"]
df_ins2g = df_ins2g[col_keep]
df_ins2g["CalvingDate"] = pd.to_datetime(df_ins2g["CalvingDate"])
df_ins2g["next_calving"] = pd.to_datetime(df_ins2g["next_calving"])
df_ins2g["CI"] = (df_ins2g["next_calving"] - df_ins2g["CalvingDate"]).dt.days
df_ins2g.drop(columns=["CalvingDate", "next_calving"], inplace=True)

df_ins2h = df_ins2f.merge(df_ins2g, on=["SE_Number", "LactationNumber"], how="left")

In [ ]:
# GL - GESTATION LENGTH
# i.e. length from last insemination to next calving
df_ins2h["InseminationDate_last"] = pd.to_datetime(df_ins2h["InseminationDate_last"])
df_ins2h["next_calving"] = pd.to_datetime(df_ins2h["next_calving"])
df_ins2h["GL"] = (df_ins2h["next_calving"] - df_ins2h["InseminationDate_last"]).dt.days
df_ins2h.to_csv("updateDF.csv", index=False)